<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/Copy_of_Freebase_Reverb_Unification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q

In [ ]:
!git clone https://github.com/castorini/BuboQA.git
%cd /content/BuboQA
!bash setup.sh 

In [ ]:
cp -r /content/BuboQA/indexes /content/drive/MyDrive

In [ ]:
cp -r /content/BuboQA/data/processed_simplequestions_dataset /content/drive/MyDrive

In [ ]:
%cd /content/drive/MyDrive/data_freebase
!unzip /content/drive/MyDrive/data_freebase/reverb_linked.zip

/content/drive/MyDrive/data_freebase
Archive:  /content/drive/MyDrive/data_freebase/reverb_linked.zip
  inflating: reverb_linked.csv       


In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
from fuzzywuzzy import fuzz

In [2]:
# mapping between MIDs and names in the form of dict['MID']=['str1', 'str2', ...,  'strN']
with open('/content/drive/MyDrive/indexes/names_2M.pkl', 'rb') as f:
    mid2name = pickle.load(f)

In [3]:
# In/Out degree for each MID in the form of dict['MID']=[In degree, Out degree]
with open('/content/drive/MyDrive/indexes/degrees_2M.pkl', 'rb') as f:
    degrees_2M = pickle.load(f)

In [4]:
# reverse mapping between string and MID in the form of dict['string']=[('MID', 'actual string', 'freebase type') ...] 
with open('/content/drive/MyDrive/indexes/entity_2M.pkl', 'rb') as f:
    entity_2M = pickle.load(f)

In [5]:
# mapping between MIDs and Relations in the form of dict['MID']=[{'fb:common.topic.notable_types', 'fb:people.person.gender', 'fb:people.person.profession'}]
with open('/content/drive/MyDrive/indexes/reachability_2M.pkl', 'rb') as f:
    reachability_2M = pickle.load(f)

In [6]:
reverb2freebace = pd.read_csv('/content/drive/MyDrive/data_freebase/reverb_linked.csv')
reverb2freebace['freebase_ID_argument1'] = reverb2freebace['freebase_ID_argument1'].apply(lambda string:'fb:m.'+str(string))
reverb2freebace['conf'] = reverb2freebace['conf'].astype(float)

In [7]:
reverb2freebace.head(4)

,ExID,arg1,rel,arg2,narg1,nrel,narg2,csents,conf,urls,reverb_no,argument1,relation_phrase,argument2,freebase_ID_argument1,freebase_entity_name,link_score,link_ambiguity_score,idk,argument1_uuid,argument2_uuid
0,625,$ 10,was deducted for,board,$ 10,be deduct for,board,1,0.95320,http://en.wikipedia.org/wiki/Danbury_and_Norwa...,0,NaN,NaN,NaN,fb:m.nan,NaN,-1.0,NaN,NaN,1e20ad13379f617635cc7f83be8cecf555567299,9ea5e97d2f481d5427902c67e55a54ec377e4761
1,705,"$ 10,000",came from,J.J.,"$ 10,000",come from,j.j.,1,0.93859,http://en.wikipedia.org/wiki/Mark_Dann,1,NaN,NaN,NaN,fb:m.nan,NaN,-1.0,NaN,NaN,fbfdbf78a3a18fbc6e48ec7fcec1cd91f978ae92,182fb1142d47f2814d7b88789f50a18d9569e93c
2,1007,$ 13.8 million,had been raised for,construction of the memorial,$ 13.8 million,have be raise for,construction of the memorial,1,0.91575,http://en.wikipedia.org/wiki/Pentagon_Memorial...,2,NaN,NaN,NaN,fb:m.nan,NaN,-1.0,NaN,NaN,7646408e5966133e148a3f77a75c37a9633cb4b8,c1d6334c2c0222da881bf3d19edf8fe67168567d
3,1112,$ 15.3 million,was paid to,Japan,$ 15.3 million,be pay to,japan,1,0.91996,http://en.wikipedia.org/wiki/Pacific_Proving_G...,3,NaN,NaN,NaN,fb:m.nan,NaN,-1.0,NaN,NaN,64deec6d2773ea99c9a09312df0eed0a73b8f98e,fcf29f6cad3232704b33e962ef5194fad3b6817b


In [8]:
for index, row in tqdm(reverb2freebace.iterrows(), total=reverb2freebace.shape[0]):
  if row['freebase_ID_argument1'] in mid2name:
    mid1 = mid = row['freebase_ID_argument1']
  else:
    mid1 = mid = row['argument1_uuid']
  mid2 = mid = row['argument2_uuid']
  reverb_string1 = str(row['arg1']).lower()
  reverb_string2 = str(row['arg2']).lower()
  relation = row['rel']
  conf = str(row['conf'])
  try:
    temp = entity_2M[reverb_string1]
  except:
    entity_2M[reverb_string1] = set()
    temp = entity_2M[reverb_string1]
  temp.add((mid1, reverb_string1, conf))
  try:
    temp = entity_2M[reverb_string2]
  except:
    entity_2M[reverb_string2] = set()
    temp = entity_2M[reverb_string2]
  temp.add((mid2, reverb_string2, conf))
  

100%|██████████| 407267/407267 [00:52<00:00, 7757.01it/s]


In [ ]:
mid_count = len(mid2name)
string_count = sum([len(name) for name in mid2name.values()])
original_relations = sum([len(relation) for relation in reachability_2M.values()])

added_mids = 0
new_entity_strings = 0
new_relations = 0

for index, row in tqdm(reverb2freebace_notmatched.iterrows(), total=reverb2freebace_notmatched.shape[0]):
  if row['reverb_no'] in questions_fact_index:
    idx = questions_fact_index.index(row['reverb_no'])
    ent_number = questions_fact_entity[idx]
    mid = row[f'argument{ent_number}_uuid']
    # print(mid)
  else:
    mid = row[f'argument1_uuid']

  added_mids+=1
  reverb_string = row['arg1']
  relation = row['rel']
  conf = str(row['conf'])
  
  mid2name[mid]=[(reverb_string, conf)]
  new_entity_strings+=1
  reachability_2M[mid]=[(relation, conf)]
  new_relations+=1
  try:
    temp = entity_2M[reverb_string]
    temp.add((mid, reverb_string, conf))
  except:
    entity_2M[reverb_string] = set([(mid, reverb_string, conf)])
  
  degrees_2M[mid]=[0, 1]


print(f'\nCombined MIDs: {mid_count}\tUnmatched (Added) MIDs: {added_mids}')
print(f'Combined Entity Strings: {string_count}\tAdded Entity Strings: {new_entity_strings}') 
print(f'Combined Relations Count: {original_relations}\tAdded Relations Count: {new_relations}') 

100%|██████████| 253913/253913 [00:40<00:00, 6232.34it/s]


Combined MIDs: 1951909	Unmatched (Added) MIDs: 253913
Combined Entity Strings: 3139797	Added Entity Strings: 253913
Combined Relations Count: 7328600	Added Relations Count: 253913


In [9]:
def get_ngram(text):
    ngram = []
    tokens = str(text).split()
    for i in range(len(tokens)+1):
        for j in range(i):
            if i-j <= 3:
                temp = " ".join(tokens[j:i])
                if temp not in ngram:
                    ngram.append(temp)
    ngram = sorted(ngram, key=lambda x: len(x.split()), reverse=True)
    return ngram

In [10]:
def get_stat_inverted_index(reverse_index):
    """
    Get the number of entry and max length of the entry (How many mid in an entry)
    """
    with open(filename, "rb") as handler:
        global  inverted_index
        inverted_index = pickle.load(handler)
        inverted_index = defaultdict(str, inverted_index)
    print("Total type of text: {}".format(len(inverted_index)))
    max_len = 0
    _entry = ""
    for entry, value in inverted_index.items():
        if len(value) > max_len:
            max_len = len(value)
            _entry = entry
    print("Max Length of entry is {}, text is {}".format(max_len, _entry))

In [11]:
from collections import defaultdict
inverted_index = defaultdict(str, entity_2M)
print("Total type of text: {}".format(len(inverted_index)))
max_len = 0
_entry = ""
for entry, value in inverted_index.items():
  if len(value) > max_len:
    max_len = len(value)
    _entry = list(value)
print("Max Length of entry is {}, text is {}".format(max_len, _entry[:100]))

Total type of text: 4936783
Max Length of entry is 249717, text is [('fb:m.071tkp', 'battir , israel', 'fb:common.topic.alias'), ('fb:m.048r0mr', 'silview , delaware', 'fb:common.topic.alias'), ('fb:m.02yxmp4', 'the gun is good. the penis is evil. the penis shoots seeds , and makes new life to poison the earth with a plague of men , as once it was. but the gun shoots death , and purifies the earth of the filth of brutals. go forth ... and kill !', 'fb:type.object.name'), ('fb:m.0f04pv6', 'music for the millions , volume 19', 'fb:type.object.name'), ('fb:m.010028', 'hebron , texas', 'fb:common.topic.alias'), ('fb:m.03fgb84', 'pillows & prayers , volume 1 & 2 : cherry red 1982-1984 ( disc 2 )', 'fb:common.topic.alias'), ('fb:m.0x8r_jx', 'jazz greats , volume 36 : the dorsey brothers : oodles of noodles', 'fb:type.object.name'), ('fb:m.0xdtb', 'eaton , new hampshire', 'fb:common.topic.alias'), ('fb:m.04rlwm', 'baekseok university , main campus', 'fb:common.topic.alias'), ('fb:m.01k7cwn', 

In [12]:
del entity_2M
# del degrees_2M
# del reachability_2M
# del mid2name

In [13]:
# test_df['order'] = range(len(test_df))
test_df = pd.read_excel('/content/drive/MyDrive/data_freebase/sbs.xlsx')
questions_fact = reverb2freebace.merge(test_df, how='inner', left_on='reverb_no', right_on='Reverb_no')
# just second entities
# questions_fact = questions_fact[questions_fact.answer_entity==2]
# just matched entities
# questions_fact = questions_fact[questions_fact.freebase_ID_argument1!='fb:m.nan']
questions_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5003 entries, 0 to 5002
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ExID                   5003 non-null   int64  
 1   arg1                   5003 non-null   object 
 2   rel                    5003 non-null   object 
 3   arg2                   5003 non-null   object 
 4   narg1                  5003 non-null   object 
 5   nrel                   5003 non-null   object 
 6   narg2                  5003 non-null   object 
 7   csents                 5003 non-null   int64  
 8   conf                   5003 non-null   float64
 9   urls                   5003 non-null   object 
 10  reverb_no              5003 non-null   int64  
 11  argument1              1924 non-null   object 
 12  relation_phrase        1924 non-null   object 
 13  argument2              1924 non-null   object 
 14  freebase_ID_argument1  5003 non-null   object 
 15  free

In [14]:
golds = []
for idx, row in questions_fact.iterrows():
  if row['freebase_ID_argument1']=='fb:m.nan':
    ans_ent = row['answer_entity']
    golds.append(row[f'argument{ans_ent}_uuid'])
  else:
    golds.append(row['freebase_ID_argument1'])


In [15]:
predicteds = questions_fact.node.astype(str).to_list()

In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import sys
# hits = []
def entity_linking(data_type, predicteds, golds, HITS_TOP_ENTITIES, output):
    stopword = set(stopwords.words('english'))
    fout = open(output, 'w')
    total = 0
    top1 = 0
    top3 = 0
    top5 = 0
    top10 = 0
    top20 = 0
    top50 = 0
    top100 = 0
    hits = []
    candidates = []
    for idx, (predicted, gold_id) in tqdm(enumerate(zip(predicteds, golds))):
        bflag = True
        total += 1
        C = []
        C_scored = []
        tokens = get_ngram(predicted)
        # print(tokens)
        if len(tokens) > 0:
            maxlen = len(tokens[0].split())
            # print(maxlen)
        for item in tokens:
            if len(item.split()) < maxlen and len(C) == 0:
                maxlen = len(item.split())
            if len(item.split()) < maxlen and len(C) > 0:
                break
            if item in stopword:
                # print('his is stopword', item)
                continue
            C.extend(inverted_index[item])
            # print(inverted_index[item])
        for mid_text_type in set(C):
            score = fuzz.ratio(mid_text_type[1], predicted.strip()) / 100.0
            C_scored.append((mid_text_type, score))
        C_scored.sort(key=lambda t: t[1], reverse=True)
        # print(C_scored[:100])
        # sys.exit()
        candidates.append(C_scored[:100])
        cand_mids = C_scored[:HITS_TOP_ENTITIES]
        for mid_text_type, score in cand_mids:
            fout.write(" %%%% {}\t{}\t{}\t{}".format(mid_text_type[0], mid_text_type[1], mid_text_type[2], score))
        fout.write('\n')
        
        midList = [x[0][0] for x in cand_mids]
        if gold_id in midList[:1]:
            top1 += 1
            if bflag:
              hits.append(1)
              bflag=False
        if gold_id in midList[:3]:
            top3 += 1
            if bflag:
              hits.append(3)
              bflag=False            
        if gold_id in midList[:5]:
            top5 += 1
            if bflag:
              hits.append(5)
              bflag=False
        if gold_id in midList[:10]:
            top10 += 1
            if bflag:
              hits.append(10)
              bflag=False
        if gold_id in midList[:20]:
            top20 += 1
            if bflag:
              hits.append(20)
              bflag=False
        if gold_id in midList[:50]:
            top50 += 1
            if bflag:
              hits.append(50)
              bflag=False
        if gold_id in midList[:100]:
            top100 += 1
            if bflag:
              hits.append(100)
              bflag=False
        if bflag:
          hits.append(-1)
          bflag=False

    print(data_type)
    print("Top1 Entity Linking Accuracy: {}".format(top1 / total))
    print("Top3 Entity Linking Accuracy: {}".format(top3 / total))
    print("Top5 Entity Linking Accuracy: {}".format(top5 / total))
    print("Top10 Entity Linking Accuracy: {}".format(top10 / total))
    print("Top20 Entity Linking Accuracy: {}".format(top20 / total))
    print("Top50 Entity Linking Accuracy: {}".format(top50 / total))
    print("Top100 Entity Linking Accuracy: {}".format(top100 / total))
    return hits, candidates

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
hits, candidates = entity_linking('test', predicteds, golds, 100, "./results")

5003it [00:24, 201.64it/s]

test
Top1 Entity Linking Accuracy: 0.3527883270037977
Top3 Entity Linking Accuracy: 0.5354787127723366
Top5 Entity Linking Accuracy: 0.5932440535678593
Top10 Entity Linking Accuracy: 0.6464121527083749
Top20 Entity Linking Accuracy: 0.6793923645812513
Top50 Entity Linking Accuracy: 0.7057765340795523
Top100 Entity Linking Accuracy: 0.7143713771736958


In [19]:
questions_fact['hit'] = hits
questions_fact['candidates'] = candidates

In [20]:
questions_fact.columns

Index(['ExID', 'arg1', 'rel', 'arg2', 'narg1', 'nrel', 'narg2', 'csents',
       'conf', 'urls', 'reverb_no', 'argument1', 'relation_phrase',
       'argument2', 'freebase_ID_argument1', 'freebase_entity_name',
       'link_score', 'link_ambiguity_score', 'idk', 'argument1_uuid',
       'argument2_uuid', 'Unnamed: 0', 'bert_tokenizer_output',
       'bert_token_ids', 'input_token_ids', 'nodes_borders', 'edges_spans',
       'node', 'edges', 'questions', 'Unnamed: 9', 'Unnamed: 0.1',
       'Meaningful', 'Reverb_no', 'Sentecne', 'Confidence', 'Question',
       'Answer', 'Frequency', 'token_matrix', 'triple', 'normalized_triple',
       'first_entity_ids', 'second_entity_ids', 'relation_ids',
       'answer_entity', 'hit', 'candidates'],
      dtype='object')

In [21]:

questions_fact[['arg1', 'rel', 'arg2', 
                'freebase_ID_argument1', 'freebase_entity_name',
                'questions', 'Answer', 'node', 'Sentecne', 'answer_entity', 'hit', 'candidates']].to_csv('/content/debug.csv')

In [22]:
from google.colab import files
files.download('/content/debug.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
questions_fact.to_csv('/content/debug.csv')

In [ ]:
questions_fact[questions_fact.hit==100]['Answer'].apply(lambda item:len(list(str(item)))).mean()

15.253521126760564

In [ ]:
all = len(questions_fact[questions_fact.hit==1])
digits = len(questions_fact[(questions_fact.hit==-1)&(questions_fact.Answer.apply(lambda item:str(item).isdigit()))])
print(digits/all)

0.056782334384858045
